# QUESTION NO:03

In [29]:
rules = [
    {
        "if": {"traffic": "high", "response_time": "slow"},
        "then": {"status": "congested"}
    },
    {
        "if": {"status": "congested", "source_type": "multiple_ips"},
        "then": {"diagnosis": "DDoS Attack"}
    },
    {
        "if": {"status": "congested", "source_type": "single_ip"},
        "then": {"diagnosis": "DoS Attack"}
    },
    {
        "if": {"connection_count": "high", "target_ports": "varied"},
        "then": {"diagnosis": "Port Scan"}
    }
]


def run_diagnosis(initial_facts):
    knowledge = initial_facts.copy()

    applied = True
    while applied:
        applied = False
        for rule in rules:
            if all(knowledge.get(k) == v for k, v in rule["if"].items()):
                for out_key, out_val in rule["then"].items():
                    if out_key not in knowledge:
                        knowledge[out_key] = out_val
                        applied = True

    return knowledge.get("diagnosis", "Unknown Threat")




scenarios = [
    {"traffic": "high", "response_time": "slow", "source_type": "multiple_ips"},
    {"traffic": "high", "response_time": "slow", "source_type": "single_ip"},
    {"connection_count": "high", "target_ports": "varied"},
    {"traffic": "normal"}
]


print(f"{'Scenario':<10} | {'Diagnosis':<20}")
print("-" * 35)

for i, logs in enumerate(scenarios, start=1):
    result = run_diagnosis(logs)
    print(f"{i:<10} | {result:<20}")

Scenario   | Diagnosis           
-----------------------------------
1          | DDoS Attack         
2          | DoS Attack          
3          | Port Scan           
4          | Unknown Threat      


# QUESTION NO :01

In [ ]:
import heapq
import random
import time
ROWS, COLS = 10, 10
START = (0, 0)
GOAL = (9, 9)
def generate_grid(prob=0.2):
    grid = [[0 if random.random() > prob else 1 for _ in range(COLS)] for _ in range(ROWS)]
    grid[START[0]][START[1]] = 0
    grid[GOAL[0]][GOAL[1]] = 0
    return grid
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])
def a_star(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_score = {start: 0}
    while open_list:
        _, current = heapq.heappop(open_list)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        for dx, dy in [(0,1),(1,0),(0,-1),(-1,0)]:
            nx, ny = current[0] + dx, current[1] + dy
            neighbor = (nx, ny)
            if 0 <= nx < ROWS and 0 <= ny < COLS and grid[nx][ny] == 0:
                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    g_score[neighbor] = tentative_g
                    f = tentative_g + heuristic(neighbor, GOAL)
                    heapq.heappush(open_list, (f, neighbor))
                    came_from[neighbor] = current
    return None
def change_environment(grid, prob=0.1):
    for _ in range(5):
        x, y = random.randint(0, 9), random.randint(0, 9)
        if (x, y) not in [START, GOAL]:
            grid[x][y] = 1 if random.random() < prob else 0
def simulate():
    success = 0
    total_length = 0
    total_replan_time = 0
    for run in range(1, 51):
        grid = generate_grid()
        current = START
        path_length = 0
        replan_time = 0
        reached = False
        while current != GOAL:
            start_time = time.time()
            path = a_star(grid, current, GOAL)
            elapsed = time.time() - start_time
            replan_time += elapsed
            if not path:
                break
            current = path[0]
            path_length += 1
            change_environment(grid)
        if current == GOAL:
            success += 1
            reached = True
        total_length += path_length
        total_replan_time += replan_time
        print(f"Run {run}:")
        print("  Goal Reached:", reached)
        print("  Path Length:", path_length)
        print("  Replanning Time:", round(replan_time, 4), "seconds")
        print("-" * 40)
    print("\nFINAL SUMMARY (50 Runs)")
    print("Success Rate:", success / 50)
    print("Average Path Length:", total_length / 50)
    print("Average Replanning Time:", total_replan_time / 50)

simulate()


Run 1:
  Goal Reached: True
  Path Length: 20
  Replanning Time: 0.0058 seconds
----------------------------------------
Run 2:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0075 seconds
----------------------------------------
Run 3:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0052 seconds
----------------------------------------
Run 4:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0047 seconds
----------------------------------------
Run 5:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0064 seconds
----------------------------------------
Run 6:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0054 seconds
----------------------------------------
Run 7:
  Goal Reached: True
  Path Length: 24
  Replanning Time: 0.0071 seconds
----------------------------------------
Run 8:
  Goal Reached: True
  Path Length: 18
  Replanning Time: 0.0038 seconds
----------------------------------------
Run 9:
  Goal Reached: True
  Pa

# QUESTION NO:02

In [5]:
class ExpertSystem:
    def __init__(self):
        self.facts = set()
        self.rules = []
        self.trace = []
    def add_rule(self, conditions, conclusion):
        self.rules.append((conditions, conclusion))
    def add_fact(self, fact):
        self.facts.add(fact)
    def infer(self):
        inferred = True
        while inferred:
            inferred = False
            for conditions, conclusion in self.rules:
                if all(c in self.facts for c in conditions) and conclusion not in self.facts:
                    self.facts.add(conclusion)
                    self.trace.append(f"Inferred {conclusion} from {conditions}")
                    inferred = True
    def ask_user(self, fact):
        ans = input(f"Is '{fact}' true? (y/n): ").lower()
        if ans == 'y':
            self.add_fact(fact)
    def diagnose(self):
        for fact in ["high_traffic", "no_user_activity", "many_ports", "unknown_files"]:
            if fact not in self.facts:
                self.ask_user(fact)
        self.infer()
        for f in self.facts:
            if f.startswith("attack_"):
                return f
        return "No threat detected"
es = ExpertSystem()
es.add_rule(["high_traffic", "no_user_activity"], "attack_DDoS")
es.add_rule(["many_ports"], "attack_Port_Scan")
es.add_rule(["unknown_files"], "attack_Malware")
test_cases = [
    {"high_traffic", "no_user_activity"},
    {"many_ports"},
    {"unknown_files"},
    {"high_traffic"},
    {"no_user_activity"}
]
correct = 0
for i, case in enumerate(test_cases):
    es.facts.clear()
    es.trace.clear()
    for fact in case:
        es.add_fact(fact)
    result = es.diagnose()
    print(f"\nTest Case {i+1}: {case}")
    print("Diagnosis:", result)
    print("Reasoning Trace:")
    for t in es.trace:
        print("-", t)
    if result != "No threat detected":
        correct += 1
print("\nAccuracy:", correct / len(test_cases))


Test Case 1: {'high_traffic', 'no_user_activity'}
Diagnosis: attack_DDoS
Reasoning Trace:
- Inferred attack_DDoS from ['high_traffic', 'no_user_activity']
- Inferred attack_Port_Scan from ['many_ports']
- Inferred attack_Malware from ['unknown_files']

Test Case 2: {'many_ports'}
Diagnosis: attack_Port_Scan
Reasoning Trace:
- Inferred attack_Port_Scan from ['many_ports']
- Inferred attack_Malware from ['unknown_files']

Test Case 3: {'unknown_files'}
Diagnosis: attack_Malware
Reasoning Trace:
- Inferred attack_Port_Scan from ['many_ports']
- Inferred attack_Malware from ['unknown_files']

Test Case 4: {'high_traffic'}
Diagnosis: attack_DDoS
Reasoning Trace:
- Inferred attack_DDoS from ['high_traffic', 'no_user_activity']

Test Case 5: {'no_user_activity'}
Diagnosis: attack_Malware
Reasoning Trace:
- Inferred attack_Malware from ['unknown_files']

Accuracy: 1.0


# QUESTION NO:4

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd



In [12]:
df = pd.read_csv("student_data.csv")


In [ ]:
X = df.drop("G3", axis=1)   
y = df["G3"]               


In [14]:
for col in X.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])


In [15]:
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [18]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [19]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)


In [20]:
svm = SVC()
svm.fit(X_train_scaled, y_train)
svm_pred = svm.predict(X_test_scaled)


In [22]:
nb = GaussianNB()
nb.fit(X_train_scaled, y_train)
nb_pred = nb.predict(X_test_scaled)


In [23]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)


In [24]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)
mlp_pred = mlp.predict(X_test_scaled)


C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [25]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)
mlp_pred = mlp.predict(X_test_scaled)


C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
importances = rf.feature_importances_
top_features = importances.argsort()[-10:]


In [27]:
X_train_top = X_train_scaled[:, top_features]
X_test_top = X_test_scaled[:, top_features]

hybrid_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
hybrid_model.fit(X_train_top, y_train)
hybrid_pred = hybrid_model.predict(X_test_top)


C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [36]:
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("MLP Accuracy:", accuracy_score(y_test, mlp_pred))
print("Hybrid Model Accuracy:", accuracy_score(y_test, hybrid_pred))


Random Forest Accuracy: 0.35443037974683544
SVM Accuracy: 0.16455696202531644
Naive Bayes Accuracy: 0.1518987341772152
Decision Tree Accuracy: 0.3291139240506329
MLP Accuracy: 0.21518987341772153
Hybrid Model Accuracy: 0.3037974683544304


In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [38]:
def evaluate_model(name, y_test, y_pred):
    print(f"\n{name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_test, y_pred, average='weighted'))
    print("F1-Score :", f1_score(y_test, y_pred, average='weighted'))


In [39]:
evaluate_model("Random Forest", y_test, rf_pred)
evaluate_model("SVM", y_test, svm_pred)
evaluate_model("Naive Bayes", y_test, nb_pred)
evaluate_model("Decision Tree", y_test, dt_pred)
evaluate_model("MLP", y_test, mlp_pred)
evaluate_model("Hybrid Model", y_test, hybrid_pred)



Random Forest
Accuracy : 0.35443037974683544
Precision: 0.32316772538291527
Recall   : 0.35443037974683544
F1-Score : 0.31553978771730656

SVM
Accuracy : 0.16455696202531644
Precision: 0.0999849306811332
Recall   : 0.16455696202531644
F1-Score : 0.1127102200869115

Naive Bayes
Accuracy : 0.1518987341772152
Precision: 0.14839977181749334
Recall   : 0.1518987341772152
F1-Score : 0.11903868952009067

Decision Tree
Accuracy : 0.3291139240506329
Precision: 0.40999095840868
Recall   : 0.3291139240506329
F1-Score : 0.3384337181805536

MLP
Accuracy : 0.21518987341772153
Precision: 0.1719911593329315
Recall   : 0.21518987341772153
F1-Score : 0.18582402126705924

Hybrid Model
Accuracy : 0.3037974683544304
Precision: 0.2630534725471434
Recall   : 0.3037974683544304


C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\cyber\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital

F1-Score : 0.2730476274780072
